In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


#__connect-googledrive__
!mkdir -p Drive
!google-drive-ocamlfuse Drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 132681 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
#__upload_file__
#solve_error[https://github.com/googlecolab/colabtools/issues/17]
# from google.colab import files
# uploaded = files.upload()
# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

Saving olist_customers_dataset.csv to olist_customers_dataset.csv
User uploaded file "olist_customers_dataset.csv" with length 9033957 bytes


In [10]:
%%bash
cd Drive/WiDS/E-commerce/
ls

olist_customers_dataset.csv
olist_geolocation_dataset.csv
olist_order_items_dataset.csv
olist_order_payments_dataset.csv
olist_order_reviews_dataset.csv
olist_orders_dataset.csv
olist_products_dataset.csv
olist_sellers_dataset.csv
product_category_name_translation.csv


In [0]:
import pandas as pd
import numpy as np
from functools import reduce

In [0]:
#__read_data__
#row_count : 99441
order_dataset = pd.read_csv("Drive/WiDS/E-commerce/olist_orders_dataset.csv")
#row_count : 99441
customer_dataset = pd.read_csv("Drive/WiDS/E-commerce/olist_customers_dataset.csv")
#row_count : 103886
order_payment_dataset = pd.read_csv("Drive/WiDS/E-commerce/olist_order_payments_dataset.csv")
#row_count : 112650
order_item_dataset = pd.read_csv("Drive/WiDS/E-commerce/olist_order_items_dataset.csv")
#row_count : 100000
order_reviews_dataset = pd.read_csv("Drive/WiDS/E-commerce/olist_order_reviews_dataset.csv")
#row_count : 32951
products_dataset = pd.read_csv("Drive/WiDS/E-commerce/olist_products_dataset.csv")
products_name_translation = pd.read_csv("Drive/WiDS/E-commerce/product_category_name_translation.csv")


In [105]:
order_reviews_dataset[order_reviews_dataset.duplicated(['order_id'])]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
1128,46abf3ea0b2710ad41390fdb79c32d84,5040757d4e06a4be96d3827b860b4e7c,5,NaN,NaN,2017-11-07 00:00:00,2017-11-10 20:07:48
3127,aa193e76d35950c4ae988237bb36ed2b,cf73e2cb1f4a9480ed70c154da3d954a,5,NaN,NaN,2018-01-18 00:00:00,2018-01-18 17:36:45
8164,40294ea5a778dc62080d6b3f55d361ce,e1bc1083cd7acd30d0576335373b907d,5,NaN,NaN,2018-03-23 00:00:00,2018-03-24 00:23:06
9128,32e2c7e889f7a185d462265398ee3631,c7cfea0c153e6382e32e84c2a9dd7d2e,5,NaN,NaN,2017-10-21 00:00:00,2017-10-22 00:47:25
9868,95a3135743556b117d888cc8c6e12e11,f9c78e6e58306dc81efbbada1ac11f24,3,NaN,O produto chegou antes do prazo mas recebi um ...,2017-09-14 00:00:00,2017-09-17 11:14:08
...,...,...,...,...,...,...,...
99762,dfb3db02188d809d5cd199496b6da87e,c0db7d31ace61fc360a3eaa34dd3457c,5,NaN,NaN,2018-02-17 00:00:00,2018-02-19 19:29:19
99883,2c6c08892b83ba4c1be33037c2842294,42ae1967f68c90bb325783ac55d761ce,4,NaN,"Chegou um pouco amassada, mas nada de mais, e ...",2017-07-03 00:00:00,2017-07-05 19:06:59
99939,2afe63a67dfd99b3038f568fb47ee761,c5334d330e36d2a810a7a13c72e135ee,5,NaN,"Muito bom, produto conforme anunciado, entrega...",2018-03-03 00:00:00,2018-03-04 22:56:47
99953,44d1e9165ec54b1d89d33594856af859,a7dbcf5043158d6fa72859eead2f3d10,4,NaN,NaN,2017-05-24 00:00:00,2017-05-24 23:15:21


In [96]:
#__payment__(因credit_card, boleto佔了9成，故將剩餘付款方式將取代為others)
#__column__(order_id, 訂單中使用boleto付款商品數, 訂單中使用cc付款商品數, 訂單中使用others付款商品數, 使用信用卡分期期數,訂單中使用boleto付款金額, 訂單中使用cc付款金額, 訂單中使用others付款金額 )

# order_payment_dataset['payment_type'].unique().tolist()

order_payment_dataset['payment_cnt'] = 1
vals_to_replace = {'credit_card':'credit_card', 'boleto':'boleto', 'voucher':'others', 'debit_card':'others', 'not_defined':'others'}
order_payment_dataset['payment_type_new'] = order_payment_dataset['payment_type'].map(vals_to_replace)
order_payment_update = order_payment_dataset[['order_id', 'payment_type_new', 'payment_installments', 'payment_value', 'payment_cnt']]
order_payment_update = order_payment_update.groupby(['order_id','payment_type_new'],as_index=False).agg({'payment_installments': 'sum','payment_value': 'sum', 'payment_cnt': 'sum'})
payment_pivot = order_payment_update.pivot_table(order_payment_update, index=['order_id'],columns='payment_type_new')
order_payment_Result = pd.DataFrame(payment_pivot.to_records())
order_payment_Result = order_payment_Result.iloc[:, [0,1,2,3,5,7,8,9]]
order_payment_Result.columns = ['order_id','payment_cnt_boleto', 'payment_cnt_creditcard', 'payment_cnt_others', 'credit_card_installments', 'payment_value_boleto', 'payment_value_creditcard', 'payment_value_others']
order_payment_Result['total_value'] = order_payment_Result.payment_value_boleto.fillna(0) + order_payment_Result.payment_value_creditcard.fillna(0) + order_payment_Result.payment_value_others.fillna(0)
order_payment_Result


# payment_Result.loc[payment_Result['order_id'] == 'ccf804e764ed5650cd8759557269dc13']

,order_id,payment_cnt_boleto,payment_cnt_creditcard,payment_cnt_others,credit_card_installments,payment_value_boleto,payment_value_creditcard,payment_value_others,total_value
0,00010242fe8c5a6d1ba2dd792cb16214,NaN,1.0,NaN,2.0,NaN,72.19,NaN,72.19
1,00018f77f2f0320c557190d7a144bdd3,NaN,1.0,NaN,3.0,NaN,259.83,NaN,259.83
2,000229ec398224ef6ca0657da4fc703e,NaN,1.0,NaN,5.0,NaN,216.87,NaN,216.87
3,00024acbcdf0a6daa1e931b038114c75,NaN,1.0,NaN,2.0,NaN,25.78,NaN,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,NaN,1.0,NaN,3.0,NaN,218.04,NaN,218.04
...,...,...,...,...,...,...,...,...,...
99435,fffc94f6ce00a00581880bf54a75a037,1.0,NaN,NaN,NaN,343.40,NaN,NaN,343.40
99436,fffcd46ef2263f404302a634eb57f7eb,1.0,NaN,NaN,NaN,386.53,NaN,NaN,386.53
99437,fffce4705a9662cd70adb13d4a31832d,NaN,1.0,NaN,3.0,NaN,116.85,NaN,116.85
99438,fffe18544ffabc95dfada21779c9644f,NaN,1.0,NaN,3.0,NaN,64.71,NaN,64.71


In [171]:
#__item__(紀錄訂單中各個商品 join出商品名稱 merge)
order_item_product = pd.merge(order_item_dataset[['order_item_id','order_id', 'product_id', 'price', 'freight_value']], products_dataset[['product_id', 'product_category_name']], on='product_id', how='left') 
order_itme_product_trans = pd.merge(order_item_product, products_name_translation, on='product_category_name', how='left') 

aggregations = {
    'order_item_id': {
        'order_item_cnt': 'max' 
    },
    'price': {
        'order_total_price': 'sum' 
    },
    'freight_value': {
        'order_freight_value': 'sum'
    },
    'product_category_name_english': {
        'order_prod_category_cnt': 'count' ,
        'order_prod_category_distint_cnt': 'nunique' 
    },
    'product_id': {
        'order_prod_id_distint_cnt': 'nunique' 
    }
}

order_itme_product_trans_update = order_itme_product_trans.groupby(['order_id']).agg(aggregations).reset_index()
order_itme_product_trans_update.columns = order_itme_product_trans_update.columns.droplevel(0)
order_itme_product_group = order_itme_product_trans.groupby('order_id')['product_category_name_english'].apply(lambda x: ','.join(x.astype(str))).reset_index()
order_itme_product_trans_update.rename(columns={ order_itme_product_trans_update.columns[0]: "order_id" }, inplace = True)
order_itme_product_trans_update.loc[order_itme_product_trans_update['order_prod_category_distint_cnt'] >2]
order_item_result = pd.merge(order_itme_product_trans_update, order_itme_product_group, on='order_id')
order_item_result['product_category_name_english'] = order_item_result['product_category_name_english'].apply(lambda x: ', '.join(sorted(set(x.split(',')))))
order_item_result

order_item_result.loc[order_item_result['order_item_cnt'] > 2]

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


,order_id,order_item_cnt,order_total_price,order_freight_value,order_prod_category_cnt,order_prod_category_distint_cnt,order_prod_id_distint_cnt,product_category_name_english
31,00143d0f86d6fbd9f9b38ab440ac16f5,3,63.99,45.30,3,1,1,sports_leisure
39,001ab0a7578dd66cd4b0a71f5b6e1e41,3,74.67,52.89,3,1,1,electronics
124,00526a9d4ebde463baee25f386963ddc,4,135.56,33.60,4,1,1,food
134,005d9a5423d47281ac463a968b3936fb,3,99.97,45.28,3,2,2,"baby, toys"
228,00a57dfbb049fbaae10763e2cf15f797,3,269.70,67.68,3,1,1,computers_accessories
...,...,...,...,...,...,...,...,...
98563,ffb8f7de8940249a3221252818937ecb,3,189.49,55.63,3,2,3,"auto, telephony"
98564,ffb9a9cd00c74c11c24aa30b3d78e03b,4,529.60,77.35,4,1,3,bed_bath_table
98600,ffd6f465cab015b452d398ac1f5d978a,3,23.52,66.18,3,1,1,christmas_supplies
98637,ffecd5a79a0084f6a592288c67e3c298,3,65.70,47.37,3,1,1,electronics


In [119]:
#__review__(同筆訂單會有多筆回覆/有回覆的標注1)
order_reviews_update = order_reviews_dataset[order_reviews_dataset.groupby('order_id').review_answer_timestamp.transform('max') == order_reviews_dataset['review_answer_timestamp']]
order_reviews_group = order_reviews_dataset.groupby('order_id')['review_comment_message'].apply(lambda x: ','.join(x.astype(str))).reset_index()
order_reviews_group.review_comment_message.replace('nan', np.nan, inplace=True)
order_reviews_group[['review_comment_message']] = order_reviews_group[['review_comment_message']].applymap(lambda x: 0 if pd.isnull(x) else 1)
order_reviews_result = pd.merge(order_reviews_update[['order_id', 'review_score', 'review_answer_timestamp']], order_reviews_group, on='order_id',how='left')
order_reviews_result

,order_id,review_score,review_answer_timestamp,review_comment_message
0,73fc7af87114b39712e6da79b0a377eb,4,2018-01-18 21:46:59,0
1,a548910a1c6147796b98fdf73dbeba33,5,2018-03-11 03:05:13,0
2,f9e4b658b201a9f2ecdecbb34bed034b,5,2018-02-18 14:36:24,0
3,658677c97b385a9be170737859d3511b,5,2017-04-21 22:02:06,1
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,2018-03-02 10:26:53,1
...,...,...,...,...
99436,22ec9f0669f784db00fa86d035cf8602,5,2017-12-11 20:06:42,0
99437,55d4004744368f5571d1f590031933e4,5,2018-03-23 09:10:43,1
99438,7725825d039fc1f0ceb7635e3f7d9206,4,2018-07-02 12:59:13,0
99439,f8bd3f2000c28c5342fedeb5e50f2e75,1,2017-12-16 01:29:43,1


In [184]:
# __join_table__
order_join_df = [order_dataset, order_item_result, order_payment_Result, order_reviews_result]
order_detail = reduce(lambda left,right: pd.merge(left,right,on='order_id',how='left'), order_join_df)
order_detail = order_detail.loc[order_detail['order_status'] == 'delivered']
replace_cols = ["payment_cnt_boleto","payment_cnt_creditcard","payment_cnt_others","credit_card_installments","payment_value_boleto","payment_value_creditcard","payment_value_others"]
order_detail[replace_cols] = order_detail[replace_cols].replace({np.nan:0})
order_detail['order_purchase_timestamp'] =  pd.to_datetime(order_detail['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S')
order_detail['order_delivered_customer_date'] =  pd.to_datetime(order_detail['order_delivered_customer_date'], format='%Y-%m-%d %H:%M:%S')
order_detail['review_answer_timestamp'] =  pd.to_datetime(order_detail['review_answer_timestamp'], format='%Y-%m-%d %H:%M:%S')
order_detail['receive_waiting_day'] = order_detail['order_delivered_customer_date'].sub(order_detail['order_purchase_timestamp'], axis=0)
order_detail['receive_waiting_day'] = order_detail['receive_waiting_day'] / np.timedelta64(1, 'D')
order_detail['review_waiting_day'] = order_detail['review_answer_timestamp'].sub(order_detail['order_delivered_customer_date'], axis=0)
order_detail['review_waiting_day'] = order_detail['review_waiting_day'] / np.timedelta64(1, 'D')



customer_update = pd.merge(customer_dataset, order_dataset[['order_id','customer_id','order_purchase_timestamp']], on='customer_id',how='left')
customer_update = customer_update[['customer_unique_id', 'customer_city', 'customer_state','order_purchase_timestamp']]
customer_update.drop_duplicates(subset=None, keep='first', inplace=True)
customer_update_result = customer_update[customer_update.groupby('customer_unique_id').order_purchase_timestamp.transform('max') == customer_update['order_purchase_timestamp']]
customer_update = customer_update[['customer_unique_id', 'customer_city', 'customer_state']]

order_customer_join = pd.merge(order_detail, customer_dataset[['customer_id','customer_unique_id']], on='customer_id',how='left')
order_customer_result = pd.merge(order_customer_join, customer_update, on='customer_unique_id',how='left')
order_customer_join

aggregations_2 = {
    'order_total_price': {
        'sum_order_total_price': 'sum',
        'avg_order_total_price': 'mean'
    },
     'order_freight_value': {
        'sum_order_freight_value': 'sum',
        'avg_order_freight_value': 'mean'
     },
    'order_prod_category_cnt': {
        'sum_order_prod_category_cnt': 'sum' ,
        'avg_order_prod_category_cnt': 'mean' 
    },
    'order_prod_category_distint_cnt': {
        'avg_order_prod_id_distint_cnt': 'mean' 
    },
    'order_prod_id_distint_cnt': {
        'avg_order_prod_id_distint_cnt': 'mean' 
    },
    'order_item_cnt': {
        'sum_order_item_cnt': 'sum' 
    },
    'payment_cnt_boleto': {
        'sum_payment_cnt_boleto': 'sum' 
    },
    'payment_cnt_creditcard': {
        'sum_payment_cnt_creditcard': 'sum' 
    },
    'payment_cnt_others': {
        'sum_payment_cnt_others': 'sum' 
    },
    'credit_card_installments': {
        'avg_credit_card_installments': 'mean' 
    },
    'payment_value_boleto': {
        'sum_payment_value_boleto': 'sum' 
    },
    'payment_value_creditcard': {
        'sum_payment_value_creditcard': 'sum' 
    },
    'payment_value_others': {
        'sum_payment_value_others': 'sum' 
    },
    'total_value': {
        'sum_total_value': 'sum' ,
        'avg_total_value': 'mean' 
    },
    'review_score': {
        'avg_review_score': 'mean' 
    },
    'order_id': {
        'order_cnt': 'nunique' 
    },
    'receive_waiting_day': {
        'avg_receive_waiting_day': 'mean' 
    },
    'review_waiting_day': {
        'avg_review_waiting_day': 'mean' 
    }
}

analysis_input_temp = order_customer_join.groupby(['customer_unique_id']).agg(aggregations_2).reset_index()
analysis_input_temp.columns = analysis_input_temp.columns.droplevel(0)
analysis_input_temp.rename(columns={ analysis_input_temp.columns[0]: "customer_unique_id" }, inplace = True)
analysis_input_temp
# analysis_input_result
# order_join_customer['customer_unique_id'].nunique()
# customer_update_result[customer_update_result.duplicated(['customer_unique_id'])]
# analysis_input_temp.loc[analysis_input_temp['customer_unique_id'] == '66980c3775537536f77b434d74e520f5']

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


,customer_unique_id,sum_order_total_price,avg_order_total_price,sum_order_freight_value,avg_order_freight_value,sum_order_prod_category_cnt,avg_order_prod_category_cnt,avg_order_prod_id_distint_cnt,avg_order_prod_id_distint_cnt,sum_order_item_cnt,sum_payment_cnt_boleto,sum_payment_cnt_creditcard,sum_payment_cnt_others,avg_credit_card_installments,sum_payment_value_boleto,sum_payment_value_creditcard,sum_payment_value_others,sum_total_value,avg_total_value,avg_review_score,order_cnt,avg_receive_waiting_day,avg_review_waiting_day
0,0000366f3b9a7992bf8c76cfdf3221e2,129.90,129.90,12.00,12.00,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,8.0,0.0,141.90,0.0,141.90,141.90,5.0,1,6.411227,4.705683
1,0000b849f77a49e4a4ce2b2a4ca5be3f,18.90,18.90,8.29,8.29,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,27.19,0.0,27.19,27.19,4.0,1,3.285590,1.185220
2,0000f46a3911fa3c0805444483337064,69.00,69.00,17.22,17.22,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,8.0,0.0,86.22,0.0,86.22,86.22,3.0,1,25.731759,1.445104
3,0000f6ccb0745a6a4b88665a16c9f078,25.99,25.99,17.63,17.63,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,4.0,0.0,43.62,0.0,43.62,43.62,4.0,1,20.037083,1.121366
4,0004aac84e0df4da2b147fca70cf8255,180.00,180.00,16.89,16.89,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,6.0,0.0,196.89,0.0,196.89,196.89,5.0,1,13.141134,4.972674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93353,fffcf5a5ff07b0908bd4e2dbc735a684,1570.00,1570.00,497.42,497.42,2.0,2.0,1.0,2.0,2.0,0.0,1.0,0.0,10.0,0.0,2067.42,0.0,2067.42,2067.42,5.0,1,27.514861,5.082002
93354,fffea47cd6d3cc0a88bd621562a9d061,64.89,64.89,19.69,19.69,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,84.58,0.0,84.58,84.58,4.0,1,30.097500,5.558090
93355,ffff371b4d645b6ecea244b27531430a,89.90,89.90,22.56,22.56,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,112.46,0.0,112.46,112.46,5.0,1,14.872083,3.096030
93356,ffff5962728ec6157033ef9805bacc48,115.00,115.00,18.69,18.69,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,5.0,0.0,133.69,0.0,133.69,133.69,5.0,1,11.858854,3.140208


In [185]:
analysis_input_temp.describe()
# order_customer_join.loc[order_customer_join['customer_unique_id'] == '66980c3775537536f77b434d74e520f5']

,sum_order_total_price,avg_order_total_price,sum_order_freight_value,avg_order_freight_value,sum_order_prod_category_cnt,avg_order_prod_category_cnt,avg_order_prod_id_distint_cnt,avg_order_prod_id_distint_cnt,sum_order_item_cnt,sum_payment_cnt_boleto,sum_payment_cnt_creditcard,sum_payment_cnt_others,avg_credit_card_installments,sum_payment_value_boleto,sum_payment_value_creditcard,sum_payment_value_others,sum_total_value,avg_total_value,avg_review_score,order_cnt,avg_receive_waiting_day,avg_review_waiting_day
count,93358.000000,93358.000000,93358.000000,93358.000000,93358.000000,93358.000000,93358.000000,93358.000000,93358.000000,93358.000000,93358.000000,93358.000000,93358.000000,93358.000000,93358.000000,93358.000000,93358.000000,93357.000000,93358.000000,93358.000000,93350.000000,93350.000000
mean,141.621480,137.508262,23.546730,22.779203,1.163671,1.123352,0.993324,1.037351,1.180370,0.205564,0.798925,0.074755,2.694177,29.670008,129.620331,5.906664,165.197003,160.316648,4.140107,1.033420,12.567474,2.900018
std,215.694014,209.860279,22.780318,21.459707,0.630592,0.539674,0.145516,0.220848,0.620857,0.418183,0.473416,0.449196,2.898408,114.787675,211.957329,45.106693,226.314012,219.572062,1.292209,0.209097,9.546171,10.870950
min,0.850000,0.850000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.590000,1.000000,1.000000,0.533414,-186.761065
25%,47.650000,46.000000,14.070000,13.880000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,25.475000,0.000000,63.052500,62.370000,4.000000,1.000000,6.788001,1.234779
50%,89.730000,86.990000,17.600000,17.240000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,2.000000,0.000000,79.330000,0.000000,107.780000,105.630000,5.000000,1.000000,10.234248,1.930029
75%,154.737500,149.900000,25.520000,24.110000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,4.000000,0.000000,158.300000,0.000000,182.557500,176.650000,5.000000,1.000000,15.717986,3.461389
max,13440.000000,13440.000000,1794.960000,1794.960000,24.000000,21.000000,3.000000,8.000000,24.000000,6.000000,15.000000,33.000000,24.000000,7274.880000,13664.080000,4445.500000,13664.080000,13664.080000,5.000000,15.000000,209.628611,518.979074
